# find tracerRNA

doveva essere CPMHNJAN_01529 ma siccome ha il contig troppo piccolo e non trovo il tracrRNA, uso un altro del gruppo87:
LDKIAEDK_00494

In [15]:
# First created Thu Jul 11 09:51:15 CEST 2019
# Made by L-F-S
# At the University Of Trento, Italy

import os
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from Bio import SeqIO
from Bio.Seq import Seq
from Bio.Alphabet import generic_dna, generic_protein, RNAAlphabet
from Bio import pairwise2


sys.path.insert(0, '/home/lorenzo.signorini/cas_mining/utils/')
import filename_discrepancies

feature="Cas9" #WARNING!!! CHANGE THIS!!
#outpath="/shares/CIBIO-Storage/CM/scratch/tmp_projects/signorini_cas/5caslocitable/out/"+feature+"/"
datadir="/shares/CIBIO-Storage/CM/scratch/tmp_projects/signorini_cas/"




In [16]:
# Import data
cas_dataset=pd.read_csv("/shares/CIBIO-Storage/CM/scratch/tmp_projects/signorini_cas/5caslocitable/known_"+feature+"_variants_table.csv", index_col=0)
cas_dataset.columns

Index(['Seq ID', 'Seq Description', 'Seq', 'Contig', 'Genome Name', 'Study',
       'Sample Name', 'SGB ID', 'pilercr_CRISPR', 'minced_CRISPR',
       'prokka_cas1', 'prokka_cas2', 'uSGB', 'Level of estimated taxonomy',
       'Estimated taxonomy'],
      dtype='object')

In [17]:
tabellazza=pd.read_csv(datadir+"/3tabellazza/crisprcas_hits_table.csv", index_col=0)
tabellazza.columns

Index(['Unnamed: 0.1', 'Genome Name', 'Study', 'Sample Name', 'Assembler',
       'Genome Size', 'N50', '# Contigs', 'Completeness', 'Contamination',
       'Strain Heterogeneity', 'SGB ID', 'pilercr_CRISPR', 'minced_CRISPR',
       'prokka_CRISPR', 'prokka_cas', 'uniref_cas'],
      dtype='object')

In [21]:
seqid ="LDKIAEDK_00494"
print("SEQ ID: ", seqid)
print(cas_dataset[cas_dataset["Seq ID"]==seqid]["Contig"].iloc[0])
print(cas_dataset[cas_dataset["Seq ID"]==seqid]["Study"].iloc[0])
print(cas_dataset[cas_dataset["Seq ID"]==seqid]["Genome Name"].iloc[0])
print(len(cas_dataset[cas_dataset["Seq ID"]==seqid]["Seq"].iloc[0]))
print(cas_dataset[cas_dataset["Seq ID"]==seqid]["minced_CRISPR"].iloc[0])
print("----------------------------------------------------------------------")


SEQ ID:  LDKIAEDK_00494
NODE_510_length_70207_cov_4.41064
XieH_2016
XieH_2016__YSZC12003_35520__bin.133
1061
[('.', '36801', '38549')]
----------------------------------------------------------------------


In [29]:
spacers=[]
repeats=[]
repeat_start_pos=[]


cas9_aa=cas_dataset[cas_dataset["Seq ID"]==seqid]["Seq"].iloc[0]
dataset=cas_dataset[cas_dataset["Seq ID"]==seqid]["Study"].iloc[0]
genomename=cas_dataset[cas_dataset["Seq ID"]==seqid]["Genome Name"].iloc[0]
SGB=cas_dataset[cas_dataset["Seq ID"]==seqid]["SGB ID"].iloc[0]
contigname=cas_dataset[cas_dataset["Seq ID"]==seqid]["Contig"].iloc[0]
print("\n-----------------------------------------------------\n"+feature+" id:\t"+seqid+"\nSGB:\t"+str(SGB)+"\nGenome Name:\t",\
      genomename+"\nContig:\t"+cas_dataset[cas_dataset["Seq ID"]==seqid]["Contig"].iloc[0],"\n-----------------------------------------------------\n")

######################################################################TODO THIS MUST BE INSIDE filename_discrepancies
# take into account filename discrepancies
if dataset.startswith("ZeeviD"):  #TODO guarda se funziona quando finisci di girare minced_runner
         s3_dataset="ZeeviD_2015"
         #genomename=filename_discrepancies.change_to_megahit(genomename)
         s3_genomename=genomename
         try:
             genomename=genomename.replace("ZeeviD_2015", "ZeeviD_2015_B")
             prokkafile_ofsample=open(annodir+"/justminced/ZeeviD_2015_B/"+genomename+".crisprcas.gff.minced")
             prokkafile_ofsample.close()
             dataset="ZeeviD_2015_B"
         except:
             genomename=genomename.replace("ZeeviD_2015_B", "ZeeviD_2015_A")
             dataset="ZeeviD_2015_A"
else:
    s3_dataset=dataset
    s3_genomename=genomename
    dataset=filename_discrepancies.s3(s3_dataset,r=True) # get working dataset name from s3 dataset name
    genomename=genomename.replace(s3_dataset, dataset)

if filename_discrepancies.dataset_has_megahit(dataset,genomename):
    genomename=filename_discrepancies.change_to_megahit(genomename)

samplename = genomename.split("__")[1]
########################################################################

# get CRISPR spacer and repeat sequence

mincedCRISPRfilename=datadir+"1crisprsearch/out/"+s3_dataset+"/"+s3_genomename+".fa.minced.out"
f=open(mincedCRISPRfilename, "r")
for line in f.readlines():
    print(line)
    if line[0]== "1" or line[0]== "2" or line[0]== "3" or line[0]== "4" or line[0]== "5" or \
    line[0]== "6" or line[0]== "7" or line[0]== "8" or line[0]== "9" or line[0]== "0":

        repeats.append(line.split("\t")[2])

        repeat_start_pos.append(line.split("\t")[0])

        if not line.split("\t")[3]=="\n":
            spacers.append(line.split("\t")[3])
f.close()

# get whole locus sequence todo this could become a function of its own, dentro utils, gli dai genomename della cas e/o seqid della cas (anzi famo solo seqid, che magari ci sn doppioni in genomi) e ti dà tt il locus

# print start and stop of CRISPR
unostranosplit=cas_dataset[cas_dataset["Seq ID"]==seqid]["minced_CRISPR"].iloc[0].split("\', \'")
crispr_start, crispr_stop = int(unostranosplit[1]), int(unostranosplit[2].strip("\')]\""))

# > get start and stop position of all Cass. questo devi metterlo nella tabella di cas non ci sono santi.

prokka_anno=tabellazza[tabellazza["Genome Name"]==s3_genomename]["prokka_cas"].iloc[0]
# so già che all casses inside here are on the same contig which is 'contig', xke le ho estratte per essere così

prokkaslist=prokka_anno.split(">")
prokkaslist.pop(0)
# get contigname of current effectorcas   
cas_position={}
for casanno in prokkaslist:
    unostranosplit=casanno.split("Name=")
    casnumber=unostranosplit[1][3] #because we know that we are loooking at 'good' loci, this is just going to be 9, 1 or 2
    unaltrostranosplit=casanno.split("\t")
    cas_start=unaltrostranosplit[3]
    cas_stop=unaltrostranosplit[4]
    cas_position["Cas"+casnumber]=[int(cas_start),int(cas_stop)]

genomefilename= "/shares/CIBIO-Storage/CM/scratch/tmp_projects/epasolli_darkmatter/allcontigs/ALLreconstructedgenomes/"+str(SGB)+"/"+genomename+".fa"
print("Retrieving locus")


# access contig:

for record in SeqIO.parse(genomefilename,"fasta"):  #c'è modo di non fare questo 'ciclo'?
    if record.id.startswith("NODE_421_length_77950_cov_4.62642"): #temp!
        #test se effettivamente la seq di cas è dove voglio io
        SeqIO.write(record, "/shares/CIBIO-Storage/CM/scratch/tmp_projects/signorini_cas/7tracrRNA/tempblastdb", "fasta")
        print("CRISPR si trova", crispr_start, crispr_stop)
        print("Cas9 si trova", cas_position["Cas9"])
        print("Cas2 si trova", cas_position["Cas2"])
        print("Cas1 si trova", cas_position["Cas1"])

        #estrai la posizione di cas9 dal contig, e traducila
        cas9start=[int(x) for x in cas_position["Cas9"]][0]
        cas9stop=[int(x) for x in cas_position["Cas9"]][1]
        cas9_nnseq=record.seq[cas9start-1:cas9stop-3] #gff should have a 1-based positional annotation (ma sto andando a occhio finchè non sono uguali, è già un oggetto Bio.Seq

        my_translated_cas9=cas9_nnseq.transcribe().translate()

  
        print(len(cas9_nnseq)/3,len(cas9_aa))


        if my_translated_cas9==cas9_aa:
            print("Cas locus on plus strand")
            plus=True
        if not my_translated_cas9==cas9_aa:
            #proviamo col reverse complement
            cas9_nnseq=record.seq[cas9start+2:cas9stop]
            my_translated_cas9=cas9_nnseq.reverse_complement().transcribe().translate()
            print("Cas locus on minus strand")
        print("Is Cas9 translated from the genome equal to the orignal annotation?", my_translated_cas9==cas9_aa)
        print(my_translated_cas9)

        #extact a locus, 200 bp a monte e a valle dei cosi che ho:
        starts, stops=[startstop[0]for startstop in cas_position.values()], [startstop[1]for startstop in cas_position.values() ]
        minimo=min([crispr_start]+starts)
        print("start: ", minimo)
        massimo=max([crispr_stop]+stops)
        print("end: ", massimo)

        #opzione 1 blast
      #1. salva np.uniq(repeats) in un fasta file: build temporary query file for blastn search

        print("let's now blast the repeat against  the genome")
        blast_folder="/shares/CIBIO-Storage/CM/scratch/tmp_projects/signorini_cas/7tracrRNA/"
        os.chdir(blast_folder)
        tempfile=open("temp_repeat_seq", "w")
        tempfile.close()
        tempfile=open("temp_repeat_seq", "a")
        unique_repeats=np.unique(repeats)
        for n, repeat in enumerate([Seq(sequence) for sequence in unique_repeats]):
            tempfile.write(">rpt"+str(n+1)+"|"+contigname+"|"+genomename+"|"+seqid+"\n"+str(repeat)+"\n") 
        tempfile.close()

        #2. Blast query file against db
        print("Running blastn of query file agianst all contigs")

        #3. make db file
        dbfile="/shares/CIBIO-Storage/CM/scratch/tmp_projects/signorini_cas/7tracrRNA/tempblastdb"
        os.system("makeblastdb -in "+dbfile+" -parse_seqids  -dbtype nucl")



        blastoutfile="/shares/CIBIO-Storage/CM/scratch/tmp_projects/signorini_cas/7tracrRNA/"\
        +str(SGB)+"__"+seqid+"__"+genomename+"__"+feature+".trcr.blastout"
        blastn_command = "blastn -out "+blastoutfile+" -outfmt \"6  qseqid sseqid pident qlen length mismatch gapopen qseq sseq sstart send evalue sstrand\" -query temp_repeat_seq -db "+dbfile+" -evalue 0.001 -word_size 11"
        print(blastn_command)
        os.system(blastn_command)
       # os.system("rm /shares/CIBIO-Storage/CM/scratch/tmp_projects/signorini_cas/7tracerRNA/temp*")



-----------------------------------------------------
Cas9 id:	LDKIAEDK_00494
SGB:	8769
Genome Name:	 XieH_2016__YSZC12003_35520__bin.133
Contig:	NODE_510_length_70207_cov_4.41064 
-----------------------------------------------------

Sequence 'NODE_421_length_77950_cov_4.62642' (77950 bp)



CRISPR 1   Range: 35373 - 36663

POSITION	REPEAT				SPACER

--------	------------------------------------	------------------------------

35373		ATTGTAGAAATCAAAGAACAGTAGTCAATCCGCAAC	TTCTGGCGTGCAAATCTTCGGCAACGCCTG	[ 36, 30 ]

35439		ATTGTAGAAATCAAAGAACAGTAGTCAATCCGCAAC	CGCGGGCGCGGATAGCCCCGCCTTTTTTGC	[ 36, 30 ]

35505		ATTGTAGAAATCAAAGAACAGTAGTCAATCCGCAAC	ATGAGCTCCTTTCCATCCTTGCCCCTAAAA	[ 36, 30 ]

35571		ATTGTAGAAATCAAAGAACAGTAGTCAATCCGCAAC	TGAAGAAGAAAGCAAATACGAAAGCAAAAT	[ 36, 30 ]

35637		ATTGTAGAAATCAAAGAACAGTAGTCAATCCGCAAC	ACCCAAGCATTATCAGTCCCGACGACTGCC	[ 36, 30 ]

35703		ATTGTAGAAATCAAAGAACAGTAGTCAATCCGCAAC	ATATGAAAGGGCTTTGAATATACCTTCCATT	[ 36, 31 ]

35770		ATTGTAGAAATCAAAGAACAGTAGTCAATCCGCAAC

In [30]:
#4. parse blast output: only print output if it is NOT one of the old repeats.
print("\n-----------------------------------------------------\nParsing BLAST output, looking for hit which are NOT the same repeats\ni.e. not starting with:", repeat_start_pos,":\n-----------------------------------------------------\n")
f=open(blastoutfile,"r")
for line in f.readlines():
    start_pos=line.split("\t")[9]
    if start_pos not in repeat_start_pos:
        if str(int(start_pos)+1) not in repeat_start_pos:
            if str(int(start_pos)-1) not in repeat_start_pos:
                print(line) 


-----------------------------------------------------
Parsing BLAST output, looking for hit which are NOT the same repeats
i.e. not starting with: ['35373', '35439', '35505', '35571', '35637', '35703', '35770', '35836', '35902', '35968', '36034', '36100', '36166', '36232', '36298', '36364', '36430', '36496', '36562', '36628', '36801', '36861', '36927', '36993', '37060', '37127', '37194', '37260', '37326', '37392', '37459', '37525', '37590', '37656', '37722', '37788', '37854', '37920', '37986', '38052', '38118', '38184', '38250', '38316', '38382', '38448', '38516'] :
-----------------------------------------------------

rpt1|NODE_510_length_70207_cov_4.41064|XieH_2016__YSZC12003_35520__bin.133|LDKIAEDK_00494	NODE_421_length_77950_cov_4.62642	100.00	36	18	0	0	ATTGTAGAAATCAAAGAA	ATTGTAGAAATCAAAGAA	20839	20822	8e-05	minus

rpt2|NODE_510_length_70207_cov_4.41064|XieH_2016__YSZC12003_35520__bin.133|LDKIAEDK_00494	NODE_421_length_77950_cov_4.62642	92.31	36	26	1	1	ATTGTAGAAATCAAAGAACGGTAGTC	

## extract sequences
ma non c'è un cazzo

In [8]:
#matched_rep_start=14242 #could just be a  crisprarray
#matched_rep_stop=14270 

In [13]:
# this is the sequence matching the repeat:
matched_rep=Seq("GTCTGGAGCCGCGCCCTTCCCCAAGGTGA") #dropped one gap

#rpts which match:
rpt1=Seq(unique_repeats[0])
rpt2=Seq(unique_repeats[1])

# since repeat is on the + strand, and this is all on the + strand, tracrRNA should be its reverse complement:
antirepeat=matched_rep.reverse_complement()
#this should go frombo to bo  = x nt  (x+1)
print(antirepeat)
len(antirepeat)

TCACCTTGGGGAAGGGCGCGGCTCCAGAC


29

In [12]:
# print terminators

f=open(blast_folder+str(SGB)+"__"+seqid+"__"+genomename+".terminators")
for line in f.readlines():
    if line.strip().startswith("6"):
        print(line)
f.close()

 625  Rnamotif - CCAAATAATCGCGTGGAAATACCGCGaTTTTTAGAAAGA -6.30



nn ce nnt manco di questi

In [17]:

terminatorend=13890-1 # (sarebe 1463-1, for indexing reasons) sarebbe la fine xke prendiamo il rev comp remembra. che ha senso xke l'antirepeat è a 14729, che è dopo
 #oppure, ce n'è uno con TTTTTTT a 14653...
terminator=Seq("GAAACAGAAAAGCCCCTGTTAAAGGGGCTTTTCTGTGCGC".upper()) # da aggiuntere .reverse_complement() se lo strand indicato è + ?
print(len(terminator))
terminator_start=terminator_end+len(terminator) #occhio che qui start e stop si riferisce al rev.compl, mentre nel antirepeat si riferisce alla direzione base

40


In [18]:
# Open genome and extract the whole sequence
# find this inside the genome, and link it to the thing found by ARNold:
for record in SeqIO.parse("/shares/CIBIO-Storage/CM/scratch/tmp_projects/signorini_cas/selected_loci/"+seqid+"/"+genomename+".fa", "fasta"):
    if record.id==contigname:
        contigseq=record.seq

#check antirepeat
print("test repeat: ", antirepeat, len(antirepeat), contigseq[matched_rep_start-1:matched_rep_stop].reverse_complement()==antirepeat) # -1 why? Indexing diverso evidentemente col blastout, vbb

# check terminator
print("test terminator: " ,terminator, len(terminator), contigseq[terminator_end:terminator_start].reverse_complement()==terminator)


pre_tracrRNA=contigseq[terminator_end:matched_rep_stop].reverse_complement()
print("pre tracrRNA:", pre_tracrRNA, len(pre_tracrRNA))

# add mario
mario=contigseq[terminator_start:matched_rep_start-1].reverse_complement()
print("mario sequence:",  mario, len(mario))  # ho fatto i check, il -1 sopra è giusto

test repeat:  TCACCTTGGGGAAGGGCGCGGCTCCAGAC 29 True
test terminator:  GAAACAGAAAAGCCCCTGTTAAAGGGGCTTTTCTGTGCGC 40 True
pre tracrRNA: TCACCTTGGGGAAGGGCGCGGCTCCAGACAAGGGGAGCCACTTAAGTGGCTTACCCGTAAAGTAACCCCCGTTCAATCTTCGGATTGGGCGGGGCGAACTTTTTTGGGCGAGTGTTAGCCAGTATTCCGTAAACGGTAGCTATGAATAGCATTTAGGCGGGTCCTTTTGAGCAATTTTATGCCTAAGGATTGATTCGATTGTGAAGAAAAGTACATAAATCTCTGTGCCGAATACGGCTCTTTTCTTGCGTGAAGCGCGACGTGCCGGCGCATTTCGTCTCGAAAAAAATGAATCGCAGATGTAGATCTGAGGTCGAATCAGTCTTCTTTTTTTGAGGCTTGAAACAGAAAAGCCCCTGTTAAAGGGGCTTTTCTGTGCGC 381
mario sequence: AAGGGGAGCCACTTAAGTGGCTTACCCGTAAAGTAACCCCCGTTCAATCTTCGGATTGGGCGGGGCGAACTTTTTTGGGCGAGTGTTAGCCAGTATTCCGTAAACGGTAGCTATGAATAGCATTTAGGCGGGTCCTTTTGAGCAATTTTATGCCTAAGGATTGATTCGATTGTGAAGAAAAGTACATAAATCTCTGTGCCGAATACGGCTCTTTTCTTGCGTGAAGCGCGACGTGCCGGCGCATTTCGTCTCGAAAAAAATGAATCGCAGATGTAGATCTGAGGTCGAATCAGTCTTCTTTTTTTGAGGCTT 312


hmmm

In [22]:
# Now, align with the rest of the stuff to see what's up
# first up, with the repeat, lets take the rpt3, which has only got 1 mismatch

alignments = pairwise2.align.localxx(rpt1, antirepeat.reverse_complement())
print(pairwise2.format_alignment(*alignments[0]))
print("rpt length:", len(rpt1))


GCCT-TG-AGCACGCA-CCCTTCCCCAAGGTGATACGCT
|  | || ||| |||  |||||||||||||||  |
G--TCTGGAGC-CGC-GCCCTTCCCCAAGGTG--A----
  Score=26

rpt length: 36


In [24]:
# align antirepat + mario with repeat
alignments = pairwise2.align.globalxx((mario).reverse_complement(), rpt1)
print(pairwise2.format_alignment(*alignments[0]))

AAGCCTCAAAAAAAGAAGACTGATTCGACCTCAGATCTACATCTGCGATTCATTTTTTTCGAGACGAAATGCGCCGGCACGTCGCGCTTCACGCAAGAAAAGAGCCGTATTCGGCACAGAGATTTATGTACTTTTCTTCACAATCGAATCAATCCTTAGGCATAAAATTGCTCAAAAGGACCCGCCTAAATGCTATTCATAGCTACCGTTTACGGAATACTGGCTAACACTCGCCCAAAAAAGTTCGCCCCGCCCAATCCGAAGATTGAACGGGGGTTACTTTACGGGTAAGCCACTTAAGTGGCTCCCCTT
                                                                                                                                                                                       | |      |   |  | |  |  |    |     ||  |    ||    |||       ||     | |||||   |  |  |        |       |    |     ||       |     | |
---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------G-C------C---T--T-G--A--G----C-----AC--G----CA----CCC-------TT-----C-CCCAA---G--G--T--------G-------A----T-----AC-------G-----C-T
  Score=36



In [25]:
# I would probably say that the tracr is just, simply la parte di pre_tracrRNA la cui antirepeat si lega alla repeat, 
# che, in questo caso, è tutta.
tracrRNA = pre_tracrRNA

print("\n----------------------------------------------------------------------")
print("         tracrRNA sequence for", feature,"protein id", seqid)
print("----------------------------------------------------------------------\n")
print("- contig name:", cas_dataset[cas_dataset["Seq ID"]==seqid]["Contig"].iloc[0])
print("- Dataset: ",cas_dataset[cas_dataset["Seq ID"]==seqid]["Study"].iloc[0])
print("- Bin: ",cas_dataset[cas_dataset["Seq ID"]==seqid]["Genome Name"].iloc[0])
print("- SGB: ",cas_dataset[cas_dataset["Seq ID"]==seqid]["SGB ID"].iloc[0])
print("----------------------------------------------------------------------\n")
print("- Length: ", len(tracrRNA),"\n- position: ", terminator_end,matched_rep_stop)
print(tracrRNA)


----------------------------------------------------------------------
         tracrRNA sequence for Cas9 protein id LJKGNKPJ_00564
----------------------------------------------------------------------

- contig name: NODE_211_length_45519_cov_7.71687
- Dataset:  KarlssonFH_2013
- Bin:  KarlssonFH_2013__S420__bin.8
- SGB:  14454
----------------------------------------------------------------------

- Length:  381 
- position:  13889 14270
TCACCTTGGGGAAGGGCGCGGCTCCAGACAAGGGGAGCCACTTAAGTGGCTTACCCGTAAAGTAACCCCCGTTCAATCTTCGGATTGGGCGGGGCGAACTTTTTTGGGCGAGTGTTAGCCAGTATTCCGTAAACGGTAGCTATGAATAGCATTTAGGCGGGTCCTTTTGAGCAATTTTATGCCTAAGGATTGATTCGATTGTGAAGAAAAGTACATAAATCTCTGTGCCGAATACGGCTCTTTTCTTGCGTGAAGCGCGACGTGCCGGCGCATTTCGTCTCGAAAAAAATGAATCGCAGATGTAGATCTGAGGTCGAATCAGTCTTCTTTTTTTGAGGCTTGAAACAGAAAAGCCCCTGTTAAAGGGGCTTTTCTGTGCGC
